---
title: "Project"
output: 
  html_document:
    pandoc_args: 
      - +RTS
      - -K64m
      - -RTS
---
##Abstract
Using devices such as Jawbone Up, Nike FuelBand, and Fitbit it is now possible to collect a large amount of data about personal activity relatively inexpensively. These type of devices are part of the quantified self movement – a group of enthusiasts who take measurements about themselves regularly to improve their health, to find patterns in their behavior, or because they are tech geeks. One thing that people regularly do is quantify how much of a particular activity they do, but they rarely quantify how well they do it. In this project, your goal will be to use data from accelerometers on the belt, forearm, arm, and dumbell of 6 participants. They were asked to perform barbell lifts correctly and incorrectly in 5 different ways. More information is available from the website here: http://groupware.les.inf.puc-rio.br/har (see the section on the Weight Lifting Exercise Dataset). 



Import Libraries

In [ ]:
%%R
library(caret)

Set seed for reproducibility

In [ ]:
%%R
set.seed(123)

#Download data 

Check if data folder exists

In [ ]:
%%R
if(!file.exists("Data")) {
  (dir.create("Data"))
}

Downlaod files.

In [ ]:
%%R
url1 <- "https://d396qusza40orc.cloudfront.net/predmachlearn/pml-training.csv"
url2  <- "https://d396qusza40orc.cloudfront.net/predmachlearn/pml-testing.csv"
file1  <- "./Data/training.csv"
file2  <- "./Data/testing.csv"


if(!file.exists(file1)) {
  download.file(url = url1, destfile=file1,method="curl")
}

if(!file.exists(file2)) {
  download.file(url = url2, destfile=file2,method="curl")
}

Read files

In [ ]:
%%R
training  <- read.csv(file1, na.strings=c("NA","#DIV/0!"))
testing  <- read.csv(file2, na.strings=c("NA","#DIV/0!"))

Traininig dataset has 160 columns and lots of missing values.. Clean the dataset by removing columns having NA values

In [ ]:
%%R
training<-training[,colSums(is.na(training)) == 0]

Divide the training dataset into train and cross validation datasets for fitting models.

In [ ]:
%%R
inTrain  <- createDataPartition(training$classe, p = 0.7, list = F)
train  <- training[inTrain, ]
cv  <- training[-inTrain, ]

Model Building 
Build the random forest mode. Use prallel processing to spped up the process.

In [ ]:
%%R
if(!file.exists('./modFit.RData')) {
  library(doMC)
  registerDoMC(cores = 16)
  modFit  <- train(classe ~., data = training, method = "rf")
  save(modFit, file='./modFit.RData')  
}

#Check the results

In [ ]:
%%R
load(file = './modFit.RData')
modFit$results

The model has reached the accuracy of `99.97836%` 


Iteration#2: Use cross validation using cross validation dataset to improve accuracy.

In [ ]:
%%R
trainControl  <- trainControl(method = "cv", number = 5)
if(!file.exists('./modFit.RData')) {
  modFitcv  <- train(classe ~ ., data = training, method="rf", trControl = trainControl)
  save(modFitcv, file = "./modFitcv.Rdata")
}

#Check the results

In [ ]:
%%R
load(file = "./modFitcv.Rdata")
modFitcv$results

The accuracy of the model is now increased to `99.99491%`


Using this model, predict the results from testing set. 



# Reference
* The data for this project come from this source: [http://groupware.les.inf.puc-rio.br/har](http://groupware.les.inf.puc-rio.br/har)